In [1]:
# Ref: https://qiita.com/noca/items/00646efd9d4a7302f522
from ipywidgets import Textarea
import io
if 'open' in globals():
    del open
if 'input' in globals():
    del input
original_open = open
class custom_open():
    def __init__(self):
        self.text = ''
    def __call__(self, file, *args, **kwargs):
        if file == 0:
            return io.StringIO(self.text)
        return original_open(file, *args, **kwargs)
    def updater(self, change):
        self.text = change["new"]
class custom_input():
    def __init__(self):
        self.__sio = io.StringIO('')
        self.shell = get_ipython()
        if self.shell.events.callbacks['pre_run_cell'] != []:
            self.shell.events.callbacks['pre_run_cell'] = []
        self.shell.events.register('pre_run_cell', self.pre_run_cell)
    def __call__(self):
        return self.__sio.readline().strip()
    def pre_run_cell(self, info):
        text = self.shell.user_ns.get('text_area', None).value
        self.__sio = io.StringIO(text)
open = custom_open()
input = custom_input()
text_area = Textarea()
text_area.observe(open.updater, names='value')
display(text_area)

Textarea(value='')

In [2]:
import sys

class SparseTable:
    def __init__(self, A):
        self.A = A
        N = len(A)
        # インデックス k として必要な最大値
        # 例: N = 8 の場合、2^2 まで計算できればよい
        max_k = (N-1).bit_length() - 1
        
        # DP: i=0,1,2,...,N-1 に対して、𝑘=0,1,...,max_k の値を順番に評価していく
        INF = 10**10
        # 𝑡𝑎𝑏𝑙𝑒[𝑖][𝑘] を初期化
        table = [[INF]*(max_k+1) for _ in range(N)]
        # table[i][k=0] = i で自明
        for i in range(N):
            table[i][0]=i
        
        # table[i][k]を求める際、table[i][k−1] の計算結果を利用する
        # さらに、table[i][k]を求める際、table[i+2^(k-1)][k−1] の計算結果を利用する 
        for k in range(1, max_k + 1):
            for i in range(N):
                if i + (1 << k) > N:
                    break
                # print(k,i,i+(1<<(k-1)))
                # k-1 で求めた結果を利用
                first = table[i][k-1]
                second = table[i+(1<<(k-1))][k-1]

                # table[i][k] の値を区間内の最小値を持つインデックスとする
                if A[first] <= A[second]:
                    table[i][k] = first
                else:
                    table[i][k] = second

        self.table = table

    # 区間[l,r] (0 <= l < r <= N-1) の最小値を持つインデックスを返す
    def query(self, l, r):
        # 同値の場合
        if l==r:
            return l
        # l > r の場合入れ替え
        if l > r:
            l, r = r, l
        # 区間の長さ
        d = r - l + 1
        k = (d-1).bit_length() - 1
        
        # A[l:r+2^k-1], A[r−2^k+1:r] の2つの区間の最小値を持つインデックスを返す
        min_idx_l = self.table[l][k]
        min_idx_r = self.table[r-(1<<k)+1][k]
        if self.A[min_idx_l] < self.A[min_idx_r]:
            return min_idx_l
        else:
            return min_idx_r
        
N=int(input())
A=list(map(lambda x: int(x)*(-1), input().split()))

ans=[-1]

if N==1:
    print(*ans)
    sys.exit(0)

# 区間最小値を事前計算
st=SparseTable(A)
for d in range(1,N):
    # 答えの左端
    l = 0
    # 答えの右端
    r = d - 1
    # 起算日が存在しない場合
    if -1*A[st.query(l,r)] <= -1*A[d]:
        ans.append(-1)
        continue
    # A[d] < A[i] を満たす最大の i を二分探索
    while l < r:
        mid = (l+r)//2
        min_index = st.query(mid,r)
        if -1*A[d] < -1*A[min_index]:
            l = min_index + 1
        else:
            r = mid - 1
        # print(d,l,r)
    # ループを抜けた時の処理を検証し、必要なら一つ戻す
    if -1*A[d] < -1*A[l]:
        ans.append(l+1)
    else:
        ans.append(l-1+1)
print(*ans)

-1 1 1 3 4 3


In [3]:
# 解説から、スタックを使って起算日を上手く保持しながら計算すれば、O(N)で解ける